# Data Cleaning

First we create a python conda environment to use for the rest of this project for reproducibility

In [25]:
!conda create -y -n wb-analysis-env -c conda-forge python=3.10 ipykernel=6.29.3 numpy=1.26.4 pandas=2.2.1 matplotlib=3.8.4 seaborn=0.13.2 wbgapi=1.0.12 requests=2.31.0 sqlalchemy=2.0.29
!python -m ipykernel install --user --name wb-analysis-env --display-name "Python (wb-analysis-env)"
!conda activate wb-analysis-env

Channels:
 - conda-forge
Platform: osx-arm64
Solving environment: done

## Package Plan ##

  environment location: /Users/Tom/miniforge3/envs/wb-analysis-env

  added / updated specs:
    - ipykernel=6.29.3
    - matplotlib=3.8.4
    - numpy=1.26.4
    - pandas=2.2.1
    - python=3.10
    - requests=2.31.0
    - seaborn=0.13.2
    - sqlalchemy=2.0.29
    - wbgapi=1.0.12


The following NEW packages will be INSTALLED:

  _openmp_mutex      conda-forge/osx-arm64::_openmp_mutex-4.5-7_kmp_llvm 
  appnope            conda-forge/noarch::appnope-0.1.4-pyhd8ed1ab_1 
  asttokens          conda-forge/noarch::asttokens-3.0.1-pyhd8ed1ab_0 
  backports.zstd     conda-forge/osx-arm64::backports.zstd-1.1.0-py310hdc7f11d_1 
  brotli             conda-forge/osx-arm64::brotli-1.2.0-h7d5ae5b_1 
  brotli-bin         conda-forge/osx-arm64::brotli-bin-1.2.0-hc919400_1 
  brotli-python      conda-forge/osx-arm64::brotli-python-1.2.0-py310h6123dab_1 
  bzip2              conda-forge/osx-arm64::bzip2-1.0.8-hd

### Imports and Setup
We import `wbgapi` to fetch data directly from the World Bank and standard libraries for file handling.

In [26]:
import wbgapi as wb
import pandas as pd
import os

### Data Retrieval
Here we define the indicators map and the specific countries we want to analyze. We then query the World Bank API for the years 2017-2023.

In [27]:
indicators = {
    'NY.GDP.PCAP.KD': 'gdp_pc',
    'SL.EMP.TOTL.SP.ZS': 'emp_ratio',
    'NY.GDP.MKTP.KD.ZG': 'gdp_growth',
    'SL.TLF.0714.ZS': 'child_labor_rate',
    'SE.PRM.UNER': 'children_out_of_school_primary'
}

# country list
countries = ['PRT','GBR','NOR','ALB','UKR','ITA']

years = range(2017, 2024)
# Download data
df = wb.data.DataFrame(indicators, economy=countries, time=years,
                       numericTimeKeys=True, labels=True, columns='series')

### Initial Data Inspection

In [28]:
df


Country  Time  NY.GDP.MKTP.KD.ZG  NY.GDP.PCAP.KD  \
economy time                                                            
ITA     2023           Italy  2023           0.715373    34146.023226   
        2022           Italy  2022           4.821177    33891.886653   
        2021           Italy  2021           8.931062    32267.709105   
        2020           Italy  2020          -8.868221    29469.798924   
        2019           Italy  2019           0.429163    32180.434102   
        2018           Italy  2018           0.826647    31819.397183   
        2017           Italy  2017           1.603700    31635.522771   
UKR     2023         Ukraine  2023           5.534734     2164.261230   
        2022         Ukraine  2022         -28.758584     1874.662964   
        2021         Ukraine  2021           3.445621     2426.612305   
        2020         Ukraine  2020          -3.752818     2324.107422   
        2019         Ukraine  2019           3.199504     2398.078369   
        2018         Ukraine  2018           3.488362     2308.666992   
        2017         Ukraine  2017           2.359972     2217.752930   
ALB     2023         Albania  2023           3.936617     5444.930001   
        2022         Albania  2022           4.826696     5178.884315   
        2021         Albania  2021           8.969576     4880.723462   
        2020         Albania  2020          -3.313756     4437.653469   
        2019         Albania  2019           2.062578     4563.467363   
        2018         Albania  2018           3.671419     4452.237147   
        2017         Albania  2017           3.283176     4283.982627   
NOR     2023          Norway  2023           0.072120    78775.849420   
        2022          Norway  2022           3.246229    79620.163546   
        2021          Norway  2021           3.908687    77812.711561   
        2020          Norway  2020          -1.278172    75287.205953   
        2019          Norway  2019           1.123747    76712.287116   
        2018          Norway  2018           0.828932    76373.647461   
        2017          Norway  2017           2.463746    76247.411645   
GBR     2023  United Kingdom  2023           0.397082    47251.438716   
        2022  United Kingdom  2022           4.839085    47682.761535   
        2021  United Kingdom  2021           8.575951    45902.833199   
        2020  United Kingdom  2020         -10.296919    42429.185800   
        2019  United Kingdom  2019           1.624475    47379.801480   
        2018  United Kingdom  2018           1.405190    46862.967359   
        2017  United Kingdom  2017           2.656505    46439.861676   
PRT     2023        Portugal  2023           2.613129    22311.305225   
        2022        Portugal  2022           6.985842    22042.868207   
        2021        Portugal  2021           5.558758    20747.699163   
        2020        Portugal  2020          -8.204632    19778.713852   
        2019        Portugal  2019           2.745685    21569.187464   
        2018        Portugal  2018           2.946282    20997.774492   
        2017        Portugal  2017           3.314721    20364.196367   

                SE.PRM.UNER  SL.EMP.TOTL.SP.ZS  SL.TLF.0714.ZS  
economy time                                                    
ITA     2023   52117.000000             46.002             NaN  
        2022   64907.000000             45.078             NaN  
        2021   55640.000000             43.842             NaN  
        2020   39912.000000             43.832             NaN  
        2019   43971.000000             44.651             NaN  
        2018   43832.000000             44.338             NaN  
        2017   46243.000000             43.952             NaN  
UKR     2023            NaN                NaN             NaN  
        2022            NaN                NaN             NaN  
        2021  299855.000000             49.266             NaN  
        2020  309449.000000 

### Sorting and Index Handling
We sort the values by Country and Time. We also reset the index to ensure 'economy' and 'time' become accessible columns rather than index levels.

In [29]:
df = df.sort_values(['Country', 'Time'])

if 'economy' not in df.columns:
    df = df.reset_index()

### Column Renaming
We apply the `indicators` dictionary to rename the World Bank codes to human-readable names. We also standardize the identifier columns.

In [30]:
# Rename indicator columns
df = df.rename(columns=indicators)

# Rename structural columns for clarity
df = df.rename(columns={'economy': 'iso_code', 'Time': 'year', 'Country': 'country_name'})

# Ensure year is an integer (removing 'YR' prefix if present)
df['year'] = df['year'].astype(str).str.replace('YR', '').astype(int)

### Handling Missing Data
We check for columns that are entirely empty. In this dataset, `child_labor_rate` is likely empty for these specific developed countries and should be removed.

In [31]:
# Drop columns that are 100% empty (NaN)
df = df.dropna(axis=1, how='all')
print("Remaining columns:", df.columns.tolist())

Remaining columns: ['iso_code', 'time', 'country_name', 'year', 'gdp_growth', 'gdp_pc', 'children_out_of_school_primary', 'emp_ratio']


### Saving Data
Finally, we reorder the columns to put identifiers first and save the clean dataset to the `../data/` directory.

In [33]:
# Reorder columns for readability
cols = ['iso_code', 'country_name', 'year'] + [c for c in df.columns if c not in ['iso_code', 'country_name', 'year']]
df = df[cols]

# Create output directory
output_dir = '../data'
os.makedirs(output_dir, exist_ok=True)

# Save to CSV
output_path = os.path.join(output_dir, 'cleaned_wb_data.csv')
df.to_csv(output_path, index=False)

print(f"Data cleaned and saved to: {output_path}")
df.head()

Data cleaned and saved to: ../data/cleaned_wb_data.csv


,iso_code,country_name,year,time,gdp_growth,gdp_pc,children_out_of_school_primary,emp_ratio
0,ALB,Albania,2017,2017,3.283176,4283.982627,1139.424549,50.152
1,ALB,Albania,2018,2018,3.671419,4452.237147,2234.391432,52.000
2,ALB,Albania,2019,2019,2.062578,4563.467363,3575.110586,53.391
3,ALB,Albania,2020,2020,-3.313756,4437.653469,10359.000000,51.026
4,ALB,Albania,2021,2021,8.969576,4880.723462,10881.000000,52.115
